In [ ]:
from pyaiml21 import Kernel
from glob import glob
import pytholog

# instanciating
bot = Kernel()
knowledge_base = pytholog.KnowledgeBase('KB')
knowledge = [
    'father(X, Y):- male(X), parent(X, Y)',
    'mother(X, Y):- female(X), parent(X, Y)',
    'child(X, Y):- parent(Y, X)'
]
knowledge_base(knowledge)

# learning aiml files
aiml_files = glob('*.aiml')       # './' because aiml files are in same folder as python file
for file in aiml_files:
    bot.learn(file)
    print(file, 'learned')

print('_'*60)
# resetting predicates
bot.respond('reset questions', 'user1')
bot.respond('reset facts', 'user1')


def set_fact(fact, value, value2=None):
    if value2:
        new_fact = fact + '(' + value.lower() + ',' + value2.lower() + ')'
    else:
        new_fact = fact + '(' + value.lower() + ')'

    knowledge.insert(0, new_fact)
    knowledge_base(knowledge)
    bot.respond('reset facts', 'user1')


def query_kb(fact, value):
    bot.respond('reset questions', 'user1')
    query = fact + '(X, ' + value.lower().strip() + ')'
    result = knowledge_base.query(pytholog.Expr(query))

    response = ''
    for value in result:
        try:
            response += value['X'].title() + ', '
        except:
            return None

    return response[:-2]


# function to check predicates
def check_predicates():
    male = bot.getPredicate('male', 'user1')
    female = bot.getPredicate('female', 'user1')
    parent = bot.getPredicate('parent', 'user1')
    child = bot.getPredicate('child', 'user1')
    father_of = bot.getPredicate('father_of', 'user1')
    mother_of = bot.getPredicate('mother_of', 'user1')
    child_of = bot.getPredicate('child_of', 'user1')

    result = None

    if male != 'unknown':
        set_fact('male', male)
    elif female != 'unknown':
        set_fact('female', female)
    elif parent != 'unknown':
        set_fact('parent', parent, child)
    elif father_of != 'unknown':
        result = query_kb('father', father_of)
    elif mother_of != 'unknown':
        result = query_kb('mother', mother_of)
    elif child_of != 'unknown':
        result = query_kb('child', child_of)

    return result

# infinite loop for bot
while True:
    response = bot.respond(input("> "), 'user1')

    result = check_predicates()
    if result:
        response = result

    print(response)
    print()
    if response.lower() == 'bye':
        break

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag

In [ ]:
def getfact(words, pos_tags, name):
    relationship_nouns = ["father", "mother", "son", "relative", "relatives", "friend", "friends"]
    noun_tags = ['NN', 'NNS', 'NNP', 'NNPS']

    current_relationship = next((word.lower() for word in words if word.lower() in relationship_nouns), None)
    final_relationship = ""
    entities = []

    if current_relationship:
        if current_relationship in ["father", "mother"]:
            final_relationship = "son_of"
        elif current_relationship in ["friend", "friends"]:
            final_relationship = "are_friends"
        elif current_relationship in ["relative", "relatives"]:
            final_relationship = "are_relatives"
        elif current_relationship == "son":
            final_relationship = "son_of"

        if pos_tags[0][0] == "My" or pos_tags[0][0] == 'I' or words[0] in name:
            entities.append(name)
            relevant_nouns = [word for word, tag in pos_tags if word not in relationship_nouns and tag in noun_tags]
            entities.append(relevant_nouns[1])
        else:
            relevant_nouns = [word for word, tag in pos_tags if word not in relationship_nouns and tag in noun_tags]
            entities = relevant_nouns[:2]
        if final_relationship == "son_of" and "my" not in words and "of" in words and words.index(current_relationship) + 1 < len(words):
            entities[0], entities[1] = entities[1], entities[0]

    if final_relationship and len(entities) > 0:
        if len(entities) == 1:
            entities.extend([name, entities[0]])
        relationship_string = f"{final_relationship}({entities[0]},{entities[1]})"
        entities.extend([final_relationship, relationship_string])

    return entities